The keras tuner is a library that helps me pickup the optimal set of hyperparameters for me Tensorflow program. The process is called hyperparameter tunig or hypertuning.
Hyperparameters are the variables tht govern the trainng process and the topology  of an ML model. These variables remain constant over the training process and directly impact the performance of our ML program. Hyperparameters are of two types:


1.   *Model Hyperparameters* which is influence model selection such as the number and width of hidden layers.
2.   *Algorithm hyperparameters* which is influence the speed qualityt of the learning algorithm such as the learning rate for Stochastic Gradient Descent(SGD) and the number of nearest neighbors fpr a K Nearest Neighbors(KNN) classifier.

In [25]:
import tensorflow as tf
from tensorflow import keras 
tf.__version__

'2.8.2'

In [26]:
pip install -q -U keras-tuner

In [27]:
import keras_tuner as kt

**Download and prepare dataset**
[fashion MMIST dataset](https://github.com/zalandoresearch/fashion-mnist)

In [28]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [29]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

**Define The Model**
When I build a model for hypertunig, I also define the hyperpararmeters search space on addition to the model architecture. The model I set up for the hypertunig is called a hypermodel.
I can define a hypermodel through two approaches:


*   By using a model builder function.
*   By subclassing the *Hypermodel* class of the Keras Tuner API



In [30]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

**Instantiate the tuner and perform hypertunig**
Instantiate the number of hypertuning. The Keras Tuner has four tuners available - *RandomSearch*, *Hyperband*, *BayesianOptimization*, and *SKlearn*.
to Instantiate the Hyperband tuner, I must specify the hypermodel, the objective to optimize and the maximum number of the epochs to the train *(max_epochs)*.

In [31]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json


INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json


INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


The Hyperband tuning algorithm uses adaptive resource allcation and early-stoping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forwards only the top-performing half of models to the next round. Hyperband determines the number of models to train in bracket by the computing 1+ log *factor* (max_epochs) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [32]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

Run the hyperparameter search. The arguments for the search method are the same as those used for the *tf.keras.model.fit* in addition to the callback above.

In [33]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 352 and the optimal learning rate for the optimizer
is 0.001.



**Train the model**
Find the optimal number of train the model with the hyperparameters obtained fron the search.

In [34]:
# Build the model with the optimal hyperparameter and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs = 50, validation_split = 0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) +1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4963 - accuracy: 0.8244 - val_loss: 0.4303 - val_accuracy: 0.8451
Epoch 2/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3701 - accuracy: 0.8657 - val_loss: 0.3865 - val_accuracy: 0.8621
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3337 - accuracy: 0.8770 - val_loss: 0.3381 - val_accuracy: 0.8776
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3078 - accuracy: 0.8865 - val_loss: 0.3404 - val_accuracy: 0.8802
Epoch 5/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2880 - accuracy: 0.8929 - val_loss: 0.3193 - val_accuracy: 0.8857
Epoch 6/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2739 - accuracy: 0.8981 - val_loss: 0.3245 - val_accuracy: 0.8831
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2624 - accuracy: 0.9016 - val_loss: 0.3199 - val_accuracy:

Re-instantiate the hypermodel and the train it with the number of epochs from above.

In [35]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs = best_epoch, validation_split = 0.2)

Epoch 1/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4988 - accuracy: 0.8240 - val_loss: 0.3929 - val_accuracy: 0.8566
Epoch 2/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3729 - accuracy: 0.8648 - val_loss: 0.3834 - val_accuracy: 0.8586
Epoch 3/29
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3321 - accuracy: 0.8779 - val_loss: 0.3460 - val_accuracy: 0.8732
Epoch 4/29
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3071 - accuracy: 0.8872 - val_loss: 0.3291 - val_accuracy: 0.8802
Epoch 5/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2883 - accuracy: 0.8925 - val_loss: 0.3191 - val_accuracy: 0.8873
Epoch 6/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2719 - accuracy: 0.8983 - val_loss: 0.3512 - val_accuracy: 0.8717
Epoch 7/29
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2602 - accuracy: 0.9022 - val_loss: 0.3485 - val_accuracy:

To finish this day, evaluate the hypermodel on the test data.

In [36]:
eval_result = hypermodel.evaluate(img_test, label_test)
print('[test_loss, test_accuracy]:', eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4491 - accuracy: 0.8892
[test_loss, test_accuracy]: [0.4491038918495178, 0.88919997215271]


The *my_dir/intro_to_kit* directory contains detailed logs and checkpoints for the trail (model configuration) run during the hyperparameter search. If the re-run hyperparameter search, the Keras Tuner uses the exising state from these logs to resume the search. To disable this behavior, pass an additional *overwrite = True* argument instantiating the tuner.